# Cleaning Data Exercises

In this exercise, we'll be returning to the American Community Survey data we used previously to measuring racial income inequality in the United States. In today's exercise, we'll be using it to measure the returns to education and how those returns vary by race and gender.




## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_missing.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}
```


### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Exercises

### Exercise 1

For these cleaning exercises, we'll return to the ACS data we've used before one last time. We'll be working with `US_ACS_2017_10pct_sample.dta`. Import the data (please use url for the autograder).

In [1]:
# Importing Required Packages
import pandas as pd
import numpy as np

# seeting default option
pd.set_option("mode.copy_on_write", True)

In [2]:
# Load Data into Dataframe and initialize results Dictionary
df = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)
results = {}

### Exercise 2

For our exercises today, we'll focus on `age`, `sex`, `educ` (education), and `inctot` (total income). Subset your data to those variables, and quickly look at a sample of 10 rows.

In [3]:
# Subsetting df on the required columns
df1 = df[["age", "sex", "educ", "inctot"]]
df1.sample(10)

,age,sex,educ,inctot
186313,51,male,grade 12,48000
207559,25,female,2 years of college,0
134604,29,male,4 years of college,1600
315042,35,male,4 years of college,89000
300585,84,female,grade 12,1800
278944,42,female,2 years of college,20800
135632,18,male,grade 11,1200
39783,63,female,grade 12,27200
227892,70,male,n/a or no schooling,25600
257891,53,male,grade 12,109500


### Exercise 3

As before, all the values of `9999999` have the potential to cause us real problems, so replace all the values of `inctot` that are `9999999` with `np.nan`. 

In [4]:
# Replacing 9999999 with NaNs in inctot column
df1["inctot"] = df1["inctot"].replace(9999999, np.nan, inplace=False)
df1.sample(10)

,age,sex,educ,inctot
60316,50,female,n/a or no schooling,0.0
214429,46,female,4 years of college,55000.0
31193,63,male,grade 12,10970.0
16797,less than 1 year old,female,n/a or no schooling,NaN
163538,63,female,grade 12,23000.0
248334,26,female,4 years of college,1000.0
22306,56,male,grade 12,50000.0
31909,less than 1 year old,female,n/a or no schooling,NaN
37210,57,female,grade 12,33550.0
103099,69,female,4 years of college,20400.0


### Exercise 4

Attempt to calculate the average age of people in our data. What do you get? Why are you getting that error?

You *should* get an error in trying to answer this question, but **PLEASE LEAVE THE CODE THAT GENERATES THIS ERROR COMMENTED OUT SO YOUR NOTEBOOK WILL RUN IN THE AUTOGRADER**. 

Then talk about the error in a markdown cell.

In [5]:
# df1["age"].mean()

## Observation
When I try to calculate the average age, I get a **TypeError** with the message ``'Categorical' with dtype category does not support reduction 'mean'`` . This seems to be happening cause the ages are stored as a categorical datatype as opposed to a numeric datatype.

### Exercise 5

We want to be able to calculate things using age, so we need it to be a numeric type. Check the current type of `age`, and look at all the values of `age` to figure out why it's categorical and not numeric. You should find two problematic categories. Store the values of these categories in `"ex5_age_young"` and `"ex5_age_old"` (once you find them, it should be clear which is which).

In [6]:
# Trying to find the entries which are causing age to be a categorical column

# create a list of the all the unique entries
age_cat = [_ for _ in df1["age"].unique()]

# loop through the list and try to convert to int, wherever error happens that is categorical
for _ in age_cat:
    try:
        int(_)
    except:
        print(
            f"One of the entries in 'age' column which is making the column categorcial is: {_}"
        )

One of the entries in 'age' column which is making the column categorcial is: less than 1 year old
One of the entries in 'age' column which is making the column categorcial is: 90 (90+ in 1980 and 1990)


In [7]:
# store these categories in results
results["ex5_age_young"] = "less than 1 year old"
results["ex5_age_old"] = "90 (90+ in 1980 and 1990)"

### Exercise 6

In order to convert `age` into a numeric variable, we need to replace those problematic entries with values that `pandas` can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values. 

**Hint 1:** Categorical variables act like strings, so you might want to use string methods! 

**Hint 2:** Remember that characters like parentheses, pluses, asterisks, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

**Hint 3:** Because the US Census has been conducted regularly for hundreds of years but exactly how the census has been conducted have occasionally changed, variables are sometimes coded in a way that might be interpreted in different ways for different census years. For example, hypothetically, one might write `90 (90+ in 1980 and 1990)` if the Censuses conducted in 1980 and 1990 used to top-code age at 90 (any values *over* 90 were just coded as 90), but more recent Censuses no longer top-coded age and recorded ages over 90 as the respondents actual age.

In [8]:
# replace the lower age with 0
df1["age"] = df1["age"].replace("less than 1 year old", "0", inplace=False)

# replace the 90+ with 100
df1["age"] = df1["age"].replace("90 (90+ in 1980 and 1990)", "90", inplace=False)

### Exercise 7

Now convert age from a categorical to numeric. Calculate the average age amoung this group, and store it in `"ex7_avg_age"`.

In [9]:
# Convert "age" column from catrgorical to numneric (int)
df1["age"] = df1["age"].astype("int")

In [10]:
results["ex7_avg_age"] = df1["age"].mean()
print(
    f"""The average age in the Dataset after making the substituitions for the categorcial values is: {results["ex7_avg_age"]:.2f} Years"""
)

The average age in the Dataset after making the substituitions for the categorcial values is: 41.30 Years


### Exercise 8

Let's now filter out anyone in our data whose age is less than 18. Note that before made `age` a numeric variable, we couldn't do this! Again, calculate the average age and this time store it in `"ex8_avg_age"`. 

Use this sample of people 18 and over for all subsequent exercises.

In [11]:
# filtering for people who are 18 years or older
df2 = df1[df1["age"] >= 18]
results["ex8_avg_age"] = df2["age"].mean()

print(
    f"""The average age of the people who are 18 years or older in the Dataset is: {results["ex8_avg_age"]:.2f} Years"""
)

The average age of the people who are 18 years or older in the Dataset is: 49.76 Years


### Exercise 9

Create an indicator variable for whether each person has *at least* a college Bachelor's degree called `college_degree`. Use this variable to calculate the number of people in the dataset with a college degree. You may assume that to get a college degree you need to complete at least 4 years of college. Save the result as `"ex9_num_college"`.

In [12]:
# make list of educ values which will count as having a college degree
col_degree = ["4 years of college", "5+ years of college"]

# create a new column to save result if people have a degree or not
df2["college_degree"] = df2["educ"].isin(col_degree)

# count people who have college degree, doing sum since True = 1
results["ex9_num_college"] = df2["college_degree"].sum()

print(
    f"""From the given dataset, {results["ex9_num_college"]} individuals have at least a college Bachelor's degree"""
)

From the given dataset, 77013 individuals have at least a college Bachelor's degree


### Exercise 10

Let's examine how the educational gender gap. Use `pd.crosstab` to create a cross-tabulation of `sex` and `college_degree`. `pd.crosstab` will give you the number of people who have each combination of `sex` and `college_degree` (so in this case, it will give us a 2x2 table with Male and Female as rows, and `college_degree` True and False as columns, or vice versa. 

In [13]:
pd.crosstab(df2.sex, df2.college_degree)

college_degree,False,True
sex,,
male,85821,36181
female,90200,40832


### Exercise 11

Counts are kind of hard to interpret. `pd.crosstab` can also normalize values to give percentages. Look at the `pd.crosstab` help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees.

Store the share (between 0 and 1) of men with college degrees in `"ex11_share_male_w_degrees"`, and the share of women with degrees in `"ex11_share_female_w_degrees"`.

In [14]:
pd.crosstab(df2.sex, df2.college_degree, normalize="index")

college_degree,False,True
sex,,
male,0.703439,0.296561
female,0.688381,0.311619


In [15]:
results["ex11_share_male_w_degrees"] = pd.crosstab(
    df2.sex, df2.college_degree, normalize="index"
).iloc[0, 1]

results["ex11_share_female_w_degrees"] = pd.crosstab(
    df2.sex, df2.college_degree, normalize="index"
).iloc[1, 1]

print(
    f"""In the given dataset, {results["ex11_share_male_w_degrees"]:%} of males have atleast a college degree as compared to {results["ex11_share_female_w_degrees"]:%} of females"""
)

In the given dataset, 29.656071% of males have atleast a college degree as compared to 31.161854% of females


### Exercise 12

Now, let's recreate that table for people who are 40 and over and people under 40. Over time, what does this suggest about the absolute difference in the share of men and women earning college degrees? Has it gotten larger, stayed the same, or gotten smaller? Store your answer (either `"the absolute difference has increased"` or `"the absolute difference has decreased"`) in `"ex12_comparing"`.

In [16]:
# create column for 40 and over bvs 40 and below
df2["over_forty"] = df2["age"] >= 40

In [17]:
ct_over_forty = pd.crosstab(
    df2[df2["over_forty"] == True].sex,
    df2[df2["over_forty"] == True].college_degree,
    normalize="index",
)
print("Comparison of college graduate proportion gender-wise for people over 40 years:")
ct_over_forty

Comparison of college graduate proportion gender-wise for people over 40 years:


college_degree,False,True
sex,,
male,0.682123,0.317877
female,0.699144,0.300856


In [18]:
ct_below_forty = pd.crosstab(
    df2[df2["over_forty"] == False].sex,
    df2[df2["over_forty"] == False].college_degree,
    normalize="index",
)
print(
    "Comparison of college graduate proportion gender-wise for people below 40 years:"
)
ct_below_forty

Comparison of college graduate proportion gender-wise for people below 40 years:


college_degree,False,True
sex,,
male,0.743143,0.256857
female,0.665710,0.334290


In [19]:
over_time = (ct_below_forty.iloc[1, 1] - ct_below_forty.iloc[0, 1]) - (
    ct_over_forty.iloc[1, 1] - ct_over_forty.iloc[0, 1]
)
results["ex12_comparing"] = "the absolute difference has increased"

In [20]:
print(
    f"Over time the absolute gap between proportion of males and females having college degree increased by {over_time:.3f}"
)

Over time the absolute gap between proportion of males and females having college degree increased by 0.094


### Exercise 13

In words, what is causing the change noted in Exercise 12 (i.e., looking at the tables above, tell me a story about Men and Women's College attainment).

## Observations
Over-time the absolute gap between the proportion of males who have a college degree and females who have a college degree has increased.
This is cause there is both an increase in the proportion of women getting a college degree and a decrease in the proportin of men getting a college degree which has increased the overall gap.

## Want More Practice?

Calculate the educational racial gap in the United States for White Americans, Black Americans, Hispanic Americans, and other groups. 

Note that to do these calculations, you'll have to deal with the fact that unlike most Americans, the American Census Bureau treats "Hispanic" not as a racial category, but a linguistic one. As a result, the racial category "White" in `race` actually includes most Hispanic Americans. For this analysis, we wish to work with the mutually exclusive categories of "White, non-Hispanic", "White, Hispanic", "Black (Hispanic or non-Hispanic)", and a category for everyone else. 

In [21]:
# verifying result Keys
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}